# Get cannibalization events

Compila cannibalization events, collecting the stores in 5 Miles radius from every seed

In [5]:
# Step 1: Set environment
# We need to set environment's correct values if we want to load project modules. 
import sys, os

# It's important load the PROJECT_ROOT path. We need to replace PROJECT_ROOT with correct folder
# Module is placed in api folder inside project root folder
# PROJECT_ROOT = '/Users/44371/Documents/Cases/V7FC/bain-vantage'
PROJECT_ROOT = '/home/jacrisol/github/bain-vantage'
MODULE_FULL_PATH = os.path.join(PROJECT_ROOT, 'api')
sys.path.insert(1, MODULE_FULL_PATH)


from api.settings import env, config

# config file sample can be foun at PROJECT_ROOT/api/config.env.sample. All keys and passwods are placed there.
# in this step, we load this file to get all carto api keys needed.
# env.read_env('/Users/44371/Documents/Cases/V7FC/bain-vantage/api/notebooks.env')
env.read_env('/home/jacrisol/github/bain-vantage/api/notebooks.env')
with env.prefixed("BAIN_VANT_API_"):
    config['carto'] = {
        'base_url': env.str('CARTO_BASE_URL', ''),
        'user': env.str('CARTO_ADMIN_USER', 'xxx'),
        'api_key': env.str('CARTO_ADMIN_API_KEY', 'xxx'),
    }
    
# Step 2: create carto variable
# create carto object to be able to use methods inside it. We import all necessary classes for the samples too 

from etl.cf_model import CartoFramesModel, GeometryType
from etl.constants.global_constants import meters_in_mile
from cartoframes import to_carto, create_table_from_query, read_carto

carto = CartoFramesModel()

In [6]:
def find_can_events(site_table, seed_table, max_miles):

    query = f"""
        WITH buffers as (
            SELECT
                cartodb_id
                , store_id
                , ST_SetSRID(ST_Buffer(the_geom::geography, 1609.34*{max_miles})::geometry, 4326) the_geom
            FROM {site_table})
            
        SELECT
            b.store_id, s.seed_id
        FROM
            buffers b
        INNER JOIN
            {seed_table} s
        ON
            ST_INTERSECTS(s.the_geom, b.the_geom)""" 
    
    return carto.get_geopandas_from_query(query)

In [7]:
def bain_method():
    events_df = find_can_events("vtg_customer_locations", "vtg_test_ws_seed_locations", 5)
    print(events_df)
    
    # Upload to carto
    to_carto(events_df, "vtg_test_ws_cann_candidate", if_exists='replace')

In [8]:
bain_method()

[2021-02-05T16:44:57Z] (271255) {carto.py:125} WARNING - Geometry column not found in the GeoDataFrame.


    store_id  seed_id
0      20740        0
1      20740       15
2      20740       18
3      10835       47
4      10410       35
..       ...      ...
134    10035       23
135    18790        0
136    18790        4
137    18790       15
138    18790       25

[139 rows x 2 columns]


[2021-02-05T16:45:00Z] (271255) {carto.py:130} INFO - Success! Data uploaded to table "vtg_test_ws_cann_candidate" correctly
